In [1]:
import keras as K
import numpy as np
import pandas as pd
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.callbacks import ModelCheckpoint

/Users/shivanikohli/Documents/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
df = pd.read_csv("Data/Gita-data.csv")

In [4]:
df.head()

,id,SA,EN
0,c:1v1,धृतराष्ट्र उवाच |धर्मक्षेत्रे कुरुक्षेत्रे समव...,"Dhritarashtra said: O Sanjay, after gathering ..."
1,c:1v2,सञ्जय उवाच ।दृष्ट्वा तु पाण्डवानीकं व्यूढं दुर...,Sanjay said: On observing the Pandava army sta...
2,c:1v3,पश्यैतां पाण्डुपुत्राणामाचार्य महतीं चमूम् ।व्...,Duryodhan said: Respected teacher! Behold the ...
3,c:1v4,अत्र शूरा महेष्वासा भीमार्जुनसमा युधि,Behold in their ranks are many powerful warrio...
4,c:1v5,अत्र शूरा महेष्वासा भीमार्जुनसमा युधि,Behold in their ranks are many powerful warrio...


In [5]:
df = pd.read_csv("Data/bible.csv")

In [6]:
df.head()

,data_id,book_id,chapter,verse,sanskrit,english
0,40001001,40,1,1,\u0907\u092c\u094d\u0930\u093e\u0939\u0940\u09...,"The book of the generation of Jesus Christ, th..."
1,40001002,40,1,2,\u0907\u092c\u094d\u0930\u093e\u0939\u0940\u09...,Abraham begat Isaac; and Isaac begat Jacob; an...
2,40001003,40,1,3,\u0924\u0938\u094d\u092e\u093e\u0926\u094d \u0...,And Judas begat Phares and Zara of Thamar; and...
3,40001004,40,1,4,\u0924\u0938\u094d\u092f \u092a\u0941\u0924\u0...,And Aram begat Aminadab; and Aminadab begat Na...
4,40001005,40,1,5,\u0924\u0938\u094d\u092e\u093e\u0926\u094d \u0...,And Salmon begat Booz of Rachab; and Booz bega...


In [7]:
train = df[:int(0.7*len(df))]
test = df[int(0.7*len(df)):]

In [8]:
trainArray = np.array(train)
testArray = np.array(test)
x_train = trainArray[:,2-1]
y_train = trainArray[:,-1]
x_test = testArray[:,2-1]
y_test = testArray[:,-1]

In [9]:
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [10]:
def max_length(lines):
    return max(len(line.split()) for line in lines)

In [11]:
eng_tokenizer = create_tokenizer(df.iloc[:,5])
eng_vocab_size = len(eng_tokenizer.word_index)+1
eng_length = max_length(df.iloc[:,5])

In [12]:
san_tokenizer = create_tokenizer(df.iloc[:,4])
san_vocab_size = len(san_tokenizer.word_index)+1
san_length = max_length(df.iloc[:,4])

In [13]:
def encode_sequences(tokenizer, length, lines):
    X = K.preprocessing.text.Tokenizer(lines)
    X = pad_sequences(X, maxlen = length, padding = 'post')
    return X

In [14]:
def encode_output(sequences, vocab_size):
    ylist = list()
    for sequence in sequences:
        encoded = to_categorical(sequence, num_classes=vocab_size)
        ylist.append(encoded)
    y = array(ylist)
    y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
    return y


In [15]:
trainX = encode_sequences(san_tokenizer, san_length, trainArray[:,2-1])
trainY = encode_sequences(eng_tokenizer, eng_length, trainArray[:,-1])
trainY = encode_output(trainY, eng_vocab_size)
# prepare validation data
testX = encode_sequences(san_tokenizer, san_length, testArray[:,2-1])
testY = encode_sequences(eng_tokenizer, eng_length,  testArray[:,-1])
testY = encode_output(testY, eng_vocab_size)

ValueError: `sequences` must be iterable.